In [17]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import re
from time import sleep
from random import randint

In [2]:
driver = webdriver.Chrome(r"C:\Users\dkg27\Downloads\chromedriver_win32\chromedriver")
driver.get(r"https://www.coursebuffet.com/areas")

In [3]:
explore_content = driver.page_source
explore_soup = BeautifulSoup(explore_content)
areas={}
for area in explore_soup.findAll('li',attrs={'class':'subname'}):
    areas[area.text]=area.find('a',href=True)['href']

In [4]:
# To load all the courses by clciking on load more button before scraping
def load_all_results():
    try:
        element = driver.find_element_by_id('load-more-button')
        while(True):
            try:
                element.click()
            except:
                print("All courses found on page")
                break
    except:
        print("All courses found on page")

In [ ]:
all_courses=[]
for sub_cat,sub_cat_url in areas.items():
    driver.get('https://www.coursebuffet.com'+sub_cat_url)
    load_all_results()
    content = driver.page_source
    soup = BeautifulSoup(content)
    for course_num in soup.findAll('div', attrs={'class' : re.compile("^search-resultunit item")}):
        y=course_num.find('a',href=True).text
        for course in course_num.findAll('span', attrs={'class':'resultlist-unit-coursetitle'}):
            courses={}
            courses['Number']=course_num.find('span').text
            x=course.find('a',href=True)
            courses['Classified as']=y
            courses['Name']=x.text
            courses['Area']=sub_cat
            courses['url']=x['href']
            driver.get('https://www.coursebuffet.com'+x['href'])
            course_content = driver.page_source
            course_soup = BeautifulSoup(course_content)
            info = course_soup.find('ul',attrs={'class':'coursepage-coursemeta'}).findAll('li')
            courses['University'] = info[1].text
            courses['Provided by'] = info[2].text
            details=course_soup.find('div',attrs={'class':"tab-content coursepage-tabcnt"})
            courses['Desc']=details.find('p').text
            details=details.findAll('li')
            courses['Instructor']=details[0].text[14:]
            courses['Extra Info']=[]
            for d in range(0,len(details)):
                courses['Extra Info'].append(details[d].text)
            all_courses.append(courses)
            sleep(randint(2,10))

In [19]:
df=pd.DataFrame(all_courses)
print(df)

       Number                                      Classified as  \
0    Arch 204                                       Architecture   
1    Arch 204                                       Architecture   
2    Arch 204                                       Architecture   
3    Arch 232                    History of Chinese Architecture   
4    Arch 250  Lower Level Architecture courses (partially cl...   
..        ...                                                ...   
337  Mrkt 408                                  Digital Marketing   
338  Mrkt 409                  Basics of Analytics and Marketing   
339  Mrkt 410                            Analytics and Marketing   
340  Mrkt 411                        Digital Marketing Landscape   
341  Mrkt 412                         Digital Marketing Planning   

                                                  Name          Area  \
0                          Architecture 101 - Part III  Architecture   
1                            Architectu